In [1]:
import ee
ee.Authenticate()
# ee.Initialize()

d:\wangari\maathai\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.11) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


True

In [2]:
ee.Initialize(project="ee-oderaisaack2")

In [3]:
makueni_point = ee.Geometry.Point([37.6203, -1.803])

makueni = makueni_point.buffer(50000).bounds()

# Visual check
print("AOI type:", makueni.type().getInfo())
print("Geometry:", makueni.getInfo())

AOI type: Polygon
Geometry: {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[37.173783848926305, -2.252770341601972], [38.06820835342738, -2.252770341601972], [38.06820835342738, -1.3530567024258997], [37.173783848926305, -1.3530567024258997], [37.173783848926305, -2.252770341601972]]]}


In [21]:
sentinel_collection = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterBounds(makueni)
    .filterDate('2015-01-01', '2025-11-22')
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
)

In [22]:
def extract_s1_safe(image):
    """
    Extract VV, VH, HH, HV (only those that exist) for each Sentinel-1 image.
    If a band is missing, it is set to None.
    No temporal averaging; each image is one row.
    """

    # List of bands to extract
    all_bands = ['VV', 'VH', 'HH', 'HV']
    
    # Get available bands in this image
    available = image.bandNames()
    

    stats = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=makueni,
        scale=10,
        bestEffort=True,
        maxPixels=1e12
    )
    
    # Extract metadata
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    orbit = image.get('orbitProperties_pass')
    rel_orbit = image.get('relativeOrbitNumber_start')
    mode = image.get('instrumentMode')

    # Initialize feature dictionary
    feature_dict = {
        'date': date,
        'orbit': orbit,
        'relative_orbit': rel_orbit,
        'instrument_mode': mode
    }

    # Add all bands safely
    for b in all_bands:
        # Check if band exists
        feature_dict[b] = ee.Algorithms.If(
            available.contains(b),
            stats.get(b),
            None
        )

    # Return as a Feature
    return ee.Feature(None, feature_dict)


In [23]:
result = sentinel_collection.map(extract_s1_safe)

In [24]:
task = ee.batch.Export.table.toDrive(
    collection=result,
    description="makss",
    fileFormat="CSV",
    fileNamePrefix="SentinelMakueni"  
)

In [20]:
task.start()
print("Export started. Check Tasks in Earth Engine.")

Export started. Check Tasks in Earth Engine.
